In [188]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np
import gspread
from oauth2client.service_account import ServiceAccountCredentials
LEN = 1807

In [189]:
URL = "https://www.tinkoff.ru/invest/stocks/?country=All&orderType=Asc&sortType=ByName&start=0&end=12"
HEADERS = {'user-agent': "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36" }

In [190]:
def get_HTML(URL, params = None):
    r = requests.get(URL, headers = HEADERS, params = params)
    return r

In [191]:
def parse(URL,Len):
    stock_params_dict = {}
    html = get_HTML(URL)
    if html.status_code == 200:
        soup = BeautifulSoup(html.text, "html.parser")
        items = soup.find_all("a", class_ = "Link-module__link__Wv1b Link-module__link_theme_default_gf5wq")
        
        stock_title_list = pd.Series([item.find('div', class_ = "Caption__caption_cyYZT") \
                         for item in items if item.find('div', class_ = "Caption__caption_cyYZT") ]).apply(lambda i: i.text)
        stock_ticker_list = pd.Series([item.find('div', class_ = "Caption__subcaption_xTAKS") \
                         for item in items if item.find('div', class_ = "Caption__subcaption_xTAKS") ]).apply(lambda i: i.text)
        stock_price_list = [price.replace("\xa0","") for price in pd.Series([item.find('div', class_ = "SecurityColumn__cellPriceSecurities_cswKs") \
                         for item in items if item.find('div', class_ = "SecurityColumn__cellPriceSecurities_cswKs") ]).apply(lambda i: i.text)]
        stock_yield_listok = pd.Series([item.find('span', class_ = "Money-module__money_UwC2N") \
                         for item in items if item.find('span', class_ = "Money-module__money_UwC2N") ]).apply(lambda i: i.text)
        stock_yield_list = [yld.replace("\xa0","") for yld in stock_yield_listok[::2]]
        for i in range(Len):
            stock_params_dict[stock_title_list[i]] = [stock_ticker_list[i],stock_price_list[i], stock_yield_list[i] ]
    else:
        print("Error")
    return  stock_params_dict


In [192]:
URL_all = "https://www.tinkoff.ru/invest/stocks/?country=All&orderType=Asc&sortType=ByName&start=0&end=12"

In [193]:
stock_params_dict  = parse(URL_all, 12)

In [194]:
all_URL = []
for i in range(1,4):
    all_URL.append("https://www.tinkoff.ru/invest/stocks/?country=All&orderType=Asc&sortType=ByName&start=0&end=" + str(12*i))

In [195]:
df = np.transpose(pd.DataFrame(stock_params_dict))  
df.columns = ["Ticker","Price","Yield"]
df["Price"] = df["Price"].map(lambda i: float(i[:-1].replace(",",".")))



In [196]:
data = parse("https://www.tinkoff.ru/invest/stocks/?country=All&orderType=Asc&sortType=ByName&start=0&end=" + str(LEN), LEN)

In [229]:
def struct(data):
    all_df = np.transpose(pd.DataFrame(data))
    all_df.columns = ["Ticker","Price","Nom_Yield"]
    all_df["Currency"] = all_df["Nom_Yield"].apply(lambda i: i[-1])
    all_df["Nom_Yield"] = all_df["Nom_Yield"].apply(lambda i: float(i.replace("+","").replace("−","-").replace(",",".")[:-1]))
    all_df["Price"] = all_df["Price"].apply(lambda i: float(i.replace(",",".")[:-1]))
    all_df["Yield_rate"] = (all_df["Price"] / (all_df["Price"] - all_df["Nom_Yield"])).apply(lambda i: round(i, 2) - 1)
    return all_df


In [230]:
df = struct(data)
credentials = ServiceAccountCredentials
scope = ['https://spreadsheets.google.com/feeds', 'https://www.googleapis.com/auth/drive']

credentials = ServiceAccountCredentials.from_json_keyfile_name('test-333300-1860a35c4df3.json', scope)

gc = gspread.authorize(credentials)

df

,Ticker,Price,Nom_Yield,Currency,Yield_rate
1-800-Flowers.com Inc,FLWS,32.20,3.92,$,0.14
10X Genomics Inc,TXG,149.92,-28.07,$,-0.16
1Life Healthcare,ONEM,17.14,0.00,$,0.00
2seventy bio,TSVT,28.26,0.00,$,0.00
2U Inc,TWOU,24.72,-14.91,$,-0.38
...,...,...,...,...,...
ЧМК,CHMK,4770.00,1660.00,₽,0.53
ЧТПЗ,CHEP,314.00,66.50,₽,0.27
Энел Россия,ENRU,0.84,0.01,₽,0.01
ЭсЭфАй,SFIN,500.20,15.40,₽,0.03


In [232]:
ws = gc.open("test").get_worksheet(0)
ws.update([test_df.columns.values.tolist()] + test_df.values.tolist())
# df[df["Ticker"] == "TGKB"]
# test_df = df.where(df.notna(), None)
# test_df[test_df["Ticker"] == "TGKB"]

{'spreadsheetId': '1XIARKYc4ANNUEXr84YZdkKAthzVKUyJQl3_1bpWe5gs',
 'updatedRange': "'Лист3'!A1:E1805",
 'updatedRows': 1805,
 'updatedColumns': 5,
 'updatedCells': 9023}